In [ ]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools

sys.path.append(os.path.abspath(os.path.join('.', '..')))
sys.path.append(os.path.abspath(os.path.join('.', 'src')))
sys.path.append(os.path.abspath(os.path.join('.', 'utils')))

# import utils.str_manip as str_manip
# import utils.feature_builder as feature_builder
import src.job_title_clustering.cluster as cluster

## Load the original data and extract features

In [ ]:
# load existing clusters
orig_df = pd.read_csv('../data/original_clusters.csv')
# keep only first 2 columns
orig_df = orig_df[['Title', 'Cluster']]

### We can use other classes to perform low-level functionality directly (for testing purposes, etc.), but the Cluster class is already subclassed.

In [ ]:
# sm = str_manip.StrManip()
# fb = feature_builder.FeatureBuilder()
cl = cluster.Cluster()

In [ ]:
# pass in the raw data to get out features
feats = cl.get_features(orig_df['Title'], True)

## Create the processing pipeline
### Configurations for all possible combinations of the processing pipeline
The kwargs for clustering algorithms are algorithm specific.
Outer arguments are used for "global" variables and pipeline building:
- scaler: choose which scaling factor to use, or None
- dim_reduce: choose which dimensionality reduction algorithm to use, or None
- clusterer: choose which clustering algorithm to use.
- n_clusters: the number of clusters to aim for during clustering

In [ ]:
cluster_config = {
    "kmeans_kwargs": {
        "init": "k-means++",
        "n_init": 50,
        "max_iter": 500,
        "random_state": 2901,
    },
    "hdbscan_kwargs": {
        'min_cluster_size': 15,
        'min_samples': 4,
        'metric': 'euclidean',
    },
    "agglom_kwargs": {
        'metric': 'euclidean',
        'linkage': 'ward',
    },
    "cluster": "kmeans", # kmeans, hdbscan, agglom
    "scaler": "minmax", # std, minmax
    "dim_reduce": "pca", # pca, umap
    "n_clusters": 7,
}

### See how the data is grouped after dimensionality reduction

In [ ]:
dim_red = cluster.Cluster()
cluster_config['dim_reduce'] = 'pca'
pipe = dim_red.setup_pipeline(**cluster_config)
pipe.fit(feats)
df = dim_red.run_pipeline(pipe, feats, orig_df['Cluster'])
dim_red.show_simple_scatterplot(df)

The above graph shows that PCA has grouped data into dense clusters of varying shapes. HDBSCAN would be a good candidate clustering algorithm for this dimensionality reduction.

In [ ]:
dim_red = cluster.Cluster()
cluster_config['dim_reduce'] = 'umap'
pipe = dim_red.setup_pipeline(**cluster_config)
pipe.fit(feats)
df = dim_red.run_pipeline(pipe, feats, orig_df['Cluster'])
dim_red.show_simple_scatterplot(df)

The above graph shows that UMAP has grouped data into more convex-looking clusters. k-means would be a good candidate clustering algorithm for this dimensionality reduction.

### Find the best number of clusters

Let's find the best number of clusters for k-means.

In [ ]:
cluster_config['cluster'] = 'kmeans'
cluster_config['dim_reduce'] = 'umap'
cl.find_best_n_components(feats, orig_df['Cluster'], cluster_config)

Looks like the best value for `n_components` could be 7.
Let's search over hyperparameters for HDBSCAN to get as close as possible to `n_components = 7`.

Running grid search for HDBSCAN...

In [ ]:
plt.figure(figsize=(6, 6))
n_rows = 5
n_cols = 5
fig, ax = plt.subplots(nrows=n_rows, ncols=n_cols, sharex=True, sharey=True, figsize=(12, 12))
subplot_i = 1
cluster_config['cluster'] = 'hdbscan'
cluster_config['dim_reduce'] = 'pca'

for i, (x, y) in enumerate(list(itertools.product(range(9, 24, 3), (range(1, 16, 3))))):
    cluster_config['hdbscan_kwargs']['min_cluster_size'] = x
    cluster_config['hdbscan_kwargs']['min_samples'] = y
    pipe = cl.setup_pipeline(**cluster_config)
    pipe.fit(feats)
    df = cl.run_pipeline(pipe, feats, orig_df['Cluster'])
    n_clusters = len(df['predicted_cluster'].unique())-1

    ax[i//n_cols][i%n_cols].plot([-1, 1], [-1.6, -1.6], color='k', lw=2)
    plt.subplot(n_rows, n_cols, subplot_i)
    plt.title(f"({x}, {y}), size {n_clusters}", fontsize='small')
    plt.scatter(df['component_1'], df['component_2'], marker='o', c=df['predicted_cluster'], s=25, edgecolors='k')

    subplot_i += 1

Looks like `min_cluster_size=15` and `min_samples=4` are good values for the hyperparameters, giving us a cluster size of 7.

In [ ]:
# set some new variables
cluster_config['n_clusters'] = 7
cluster_config['hdbscan_kwargs']['min_cluster_size'] = 15
cluster_config['hdbscan_kwargs']['min_samples'] = 4

In [ ]:
cluster_config['cluster'] = 'hdbscan'
cluster_config['dim_reduce'] = 'pca'
cl.find_best_n_components(feats, orig_df['Cluster'], cluster_config)

We can see that performance is fairly good at 7 clusters for HDBSCAN as well. Even though performance increases with more components, I want to keep analysis of cluster simpler for now.

### Choose a clustering algorithm and set up the pipeline

We start with k-means + UMAP.

In [ ]:
cluster_config['cluster'] = 'kmeans'
cluster_config['dim_reduce'] = 'umap'
pipe = cl.setup_pipeline(**cluster_config)
pipe["dim_reducer"][cluster_config['dim_reduce']].n_components = 2

pipe.fit(feats)

kmeans_df = cl.run_pipeline(pipe, feats, orig_df['Cluster'])
kmeans_labels = cl.pred_lbl

### Use the best values to create clusters

In [ ]:
cl.show_scatterplot(kmeans_df)

Now we do the same for HDBSCAN + PCA.

In [ ]:
cluster_config['cluster'] = 'hdbscan'
cluster_config['dim_reduce'] = 'pca'
pipe = cl.setup_pipeline(**cluster_config)
pipe["dim_reducer"][cluster_config['dim_reduce']].n_components = 2

pipe.fit(feats)

hdbscan_df = cl.run_pipeline(pipe, feats, orig_df['Cluster'])
hdbscan_labels = cl.pred_lbl

In [ ]:
cl.show_scatterplot(hdbscan_df)

## Analyze and create new clusters

The following graphs show top extracted words using NMF, PLSA, and LDA. These top words can be used to create or enhance clusters.

In [ ]:
top_words = cl.topic_analysis(orig_df['Title'], feats, cluster_config['n_clusters'], n_top_words=15)

We can use LLMs to help us categorize these groups of top words. The output of the LLM will not be directly used as a cluster definition, but rather as a starting point for generating ideas about how to define the clusters.

In [ ]:
import ollama

categories = {}
instruction_prompt = "You are a helpful chatbot. Use only the following pieces of context to answer the question. Don't make up any new information:\n"
user_prompt = "Categorize the following group of data into one category, using as few words as possible:\n"

for model, data in top_words.items():
    categories[model] = []

    for d in data:
        response = ollama.chat(
            model='hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF',
            messages=[
                {'role': 'system', 'content': instruction_prompt},
                {'role': 'user', 'content': user_prompt + ','.join(d)},
            ],
        )
        categories[model].append(response['message']['content'])

In [ ]:
categories

Taking these categories, together with the top words, we can define our own clusters. These cluster definitions and their associated top words can then be used in a KNN algorithm to assign new clusters to our original data.

In [ ]:
curated_categories = {
    'nmf': [
        'Marketing', # Marketing Management
        'Product Management', # Customer/Analyst/Marketing.
        'IT Engineering', # Technology or IT
        'Data Analytics', # Analysts.
        'Project Management', # Product/Project
        'IT Engineering', # management
        'Data Analytics', # Data Engineer.
    ],
    'plsa': [
        'Marketing', # Project management.
        'Product Management', # Product Management.
        'Business Strategy', # Company
        'Data Analytics', # Data Analyst/Developer
        'Project Management', # Companies.
        'IT Engineering', # Leadership.
        'Business Strategy', # Management team.
    ],
    'lda': [
        'Data Analytics', # Data Analyst.
        'Business Strategy', # Business Management Team
        'IT Engineering', # Company
        'Project Management', # Technical
        'Customer Experience', # CEO.
        'Data Analytics', # Data.
        'IT Engineering', # Leadership roles.
    ]
}

First, we compile the top words from our topic extraction into feature vectors.

In [ ]:
top_words_by_cat = {}
top_cats = set()
# remove words that appear across multiple topics
# this is a hand-curated list of words that are ambiguous or possibly semantically vacuous in this context
remove_words = [
    'executive', 'vice', 'manager', 'senior', 'head', 'director', 'president', 'chief'
]
total_categories = 0

# create a list of words under each new curated category based on the extracted words from each topic extraction model
for model, cats in curated_categories.items():
    for i, cat in enumerate(cats):
        if cat not in top_words_by_cat:
            top_words_by_cat[cat] = set()
            top_cats.update([cat])
            total_categories += 1
        
        top_words_by_cat[cat].update([w for w in top_words[model][i] if w not in remove_words])

top_cats = list(top_cats)
top_cats.sort()
top_words_feats = np.zeros((len(cl.vocab), total_categories))

# create a feature vector of the top words based on the vocabulary from the original data
for i in range(total_categories):
    curr_cat = top_cats[i]

    for word in top_words_by_cat[curr_cat]:
        top_words_feats[cl.vocab.index(word)][i] = 1

top_words_feats = top_words_feats.T

Then, we scale the features and run dimensionality reduction (here I am using PCA).

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
pca = PCA(n_components=2, random_state=2901)

top_words_feats = pca.fit_transform(top_words_feats)
top_words_feats = scaler.fit_transform(top_words_feats)
top_word_df = pd.DataFrame(top_words_feats)

The scatterplot shows where the centroid of each category resides.

In [ ]:
plt.scatter(x=top_word_df[0], y=top_word_df[1])

In [ ]:
new_df = orig_df.copy()
new_df['new_cluster'] = 'Other'
new_feats = cl.run_dim_reducer(pipe, feats)

Finally, we can set up the KNN classifier. I tried out different values for `n_neighbors`, `weights`, and `metric` that eventually gave me some reasonable results.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, weights='distance')

knn.fit(top_words_feats, top_cats)

In [ ]:
for idx, point in enumerate(new_feats):
    pred_lbl = knn.predict([tuple(point)])[0]
    new_df['new_cluster'].at[idx] = pred_lbl

Save the new clusters to a CSV file.

In [ ]:
new_df.to_csv('../data/new_clusters.csv')

In [ ]:
# load existing clusters
new_df = pd.read_csv('../data/new_clusters.csv')
# keep only 2 columns
new_df = new_df[['Title', 'new_cluster']]

ncl = cluster.Cluster()
# pass in the raw data to get out features
nfeats = ncl.get_features(new_df['Title'], True)

Here are a couple of graphs showing the performance of k-means and HDBSCAN over the data now.

In [ ]:
cluster_config['cluster'] = 'kmeans'
cluster_config['dim_reduce'] = 'umap'
ncl.find_best_n_components(nfeats, new_df['new_cluster'], cluster_config)

In [ ]:
cluster_config['cluster'] = 'hdbscan'
cluster_config['dim_reduce'] = 'pca'
ncl.find_best_n_components(nfeats, new_df['new_cluster'], cluster_config)

Both graphs look like there are much better results this time!

We can also look at the clustering results again.

In [ ]:
cluster_config['cluster'] = 'kmeans'
cluster_config['dim_reduce'] = 'umap'
new_pipe = ncl.setup_pipeline(**cluster_config)
new_pipe["dim_reducer"][cluster_config['dim_reduce']].n_components = 2

new_pipe.fit(nfeats)

new_kmeans_df = ncl.run_pipeline(new_pipe, nfeats, new_df['new_cluster'])

In [ ]:
ncl.show_scatterplot(new_kmeans_df)

In [ ]:
cluster_config['cluster'] = 'hdbscan'
cluster_config['dim_reduce'] = 'pca'
new_pipe = ncl.setup_pipeline(**cluster_config)
new_pipe["dim_reducer"][cluster_config['dim_reduce']].n_components = 2

new_pipe.fit(nfeats)

new_hdbscan_df = ncl.run_pipeline(new_pipe, nfeats, new_df['new_cluster'])

In [ ]:
ncl.show_scatterplot(new_hdbscan_df)

It looks like the clusterings are more homogeneous for Data Analytics and IT Engineering, but additional work needs to be done to better define the other topics. Next steps could be to investigate the clusters for patterns, and/or to involve subject-matter experts to refine definitions. We can also try out different cluster sizes and number of clusters, given that the definition and distribution of the data may have changed during analysis.